# 2. Using `yadg` to extract (meta)data from single files

Since `yadg-5.0`, it is possible to access the file parsing functionality of several of the implemented filetypes, using the `yadg extract` command. An up-to-date list of exposed *Extractors* is shown in the [**yadg extractors** section of the sidebar](https://dgbowl.github.io/yadg/5.1/index.html) in the project documentation. In this tutorial, we will show you how to use yadg on the command line to extract the data from a BioLogic `.mpr` file, and the meta-data from a PANalytical `.xrdml` file.

## 2.1 `yadg extract` usage

The usage of the `yadg extract` command can be accesed via the `--help` argument:

In [ ]:
!yadg extract --help

## 2.2 Extracting data from a BioLogic `.mpr` file
As of `yadg-5.1`, any data from the supported `filetypes` is extracted into a [`DataTree` object](https://xarray-datatree.readthedocs.io/en/latest/generated/datatree.DataTree.html#datatree.DataTree), which is stored as a NetCDF file (`.nc`). As shown in the above usage, to extract the data, we need to specify the `filetype` (in this case `eclab.mpr`) and the input file (using the `infile` argument). The output file path `[outfile]` is optional and will default to the basename of the `infile` with a `.nc` suffix.

In this example, we are extracting some PEIS data from the `extract/peis.mpr` file. As we don't specify an `outfile`, the data will be dumped into `extract/peis.nc`:

In [ ]:
!yadg extract eclab.mpr extract/peis.mpr

We can open the generated `extract/peis.nc` NetCDF file using the `datatree` package. The *Extractors* exposed in `yadg-5.1` always write a `DataTree` into the resulting NetCDF file, therefore the data can be loaded using the following commands:

In [ ]:
from datatree import open_datatree
dt = open_datatree('extract/peis.nc')
dt

Note that the above `DataTree` contains only the root node, which can be also accessed as `dt["/"]`. This node contains a `xarray.Dataset`, with one dimension (`uts`, corresponding to the unix timestamp, with shape `(32, )`), 48 data variables (including nominal values and standard errors), and some metadata stored in the attributes section, including the `original_metadata` extracted from the file and serialised as a json string, as well as yadg-specific provenance metadata including the creation date and command used.

We can access the data within the `DataTree` using the usual dictionary notation, returning a `xarray.DataArray` object:

In [ ]:
dt["freq"]

Note that the `xarray.DataArray` contains a reference to the coordinates (`uts`), and its annotations include `units` (here `Hz`) and a reference to the standard error of the data points via `ancillary_variables` (here `freq_std_err`), following the [NetCDF Climate and Forecast Metadata Conventions](https://cfconventions.org/Data/cf-conventions/cf-conventions-1.10/cf-conventions.pdf). For more information about working with `xarray` objects, see the [`xarray` documentation](https://docs.xarray.dev/en/stable/index.html).

# 2.3 Extracting metadata from a PANalytical `.xrdml` file
In some cases, it might be more conventient to extract only the metadata from a file, e.g. in cases where transferring the whole dataset over the network is inconvenient. According to the usage instructions, this can be achieved using the `--meta-only` switch. In this case, the output file will be a JSON. 

To extract data from PANalytical `.xrdml` files, we need to pass `panalytical.xrdml` as the `filetype` argument to yadg:

In [ ]:
!yadg extract --meta-only panalytical.xrdml extract/scans.xrdml extract/scans-metadata.json

As the output file in [`extract/scans-metadata.json`](extract/scans-metadata.json) is a JSON file, we can view it by opening it in Firefox, or using `IPython.display.JSON`:

In [ ]:
from IPython.display import JSON
JSON(filename = "extract/scans-metadata.json", expanded=False)

As the returned object is a `DataTree`, the metadata object generated from this `DataTree` is also a tree. In this case, it contains only one node, `"/"`. This node contains the following entries:

- `"attrs"`: containing the `original_metadata` and the yadg provenance info;
- `"dims"`: the data dimensions, listing the dimension names and their shapes, in this case `uts` and `angle`;
- `"coords"`: the data coordinates, including the coordinate names (also `uts` and `angle`), which dimensions these coordinates represent, and their attributes, shape, and dtype,
- `"data_vars"`: the data variables, including the variable names (here `intensity` as well as the ancillary variables `intensity_std_err` and `angle_std_err`), which dimensions apply to these variables, and their attributes, shape, and dtype.


[Back to index](index.ipynb)